# Part 2: Modeling & Evaluation

## Imports

In [ ]:
import pandas                as pd
import numpy                 as np
import matplotlib.pyplot     as plt
import seaborn               as sns
from sklearn.ensemble        import RandomForestClassifier
from sklearn.linear_model    import LogisticRegression
from sklearn.svm             import SVC
from xgboost                 import XGBClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics         import roc_auc_score
from sklearn.metrics         import accuracy_score
from sklearn.metrics         import precision_score
from sklearn.metrics         import recall_score
from sklearn.metrics         import f1_score
from sklearn.metrics         import balanced_accuracy_score
from sklearn.pipeline        import Pipeline
from IPython.core.display    import display, HTML
from IPython.display         import display_html
sns.set(style = "white", palette = "deep")
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline

## Table Of Contents

1. [Reading In The Data](#Reading-In-The-Data)
    - [Overview](#Overview)
    
    
2. [Feature Engineering](#Feature-Engineering)

## Reading In The Data

In [ ]:
pulsar = pd.read_csv("../Data/pulsar_cleaned.csv")

### Overview

In [ ]:
# Checking the head of the data

pulsar.head()

In [ ]:
# Checking the shape of the data

print(f"The shape of the dataset is: {pulsar.shape}")

In [ ]:
# Summary of column data types

pulsar.dtypes.value_counts()

In [ ]:
# Checking for null values

pulsar.isnull().sum()

## Feature Engineering

- square `mean_ip`

- square `sd_ip`

-----

- multiply `mean_ip` x `sd_ip` 

- multiply `ex_kurt_ipd` x `skew_ip`

- multiply `mean_dmnsr` x `dmsnr` 

- multiply `ex_kurt_dmsnr` x `skew_dmsnr`
